# 穴あきバケツのシミュレーション

制御技術を使うと、時間とともに状態が変化するシステムを、いい感じの状態に持って行ったり、あるいはいい感じの状態にキープしたりすることができます。
たとえばロボットの姿勢を操作して倒れないようにしたり、エアコンの出力を操作して部屋の温度を快適にしたりするために、制御は使われています。

制御の例を見るために、実機を毎回出してくるわけにはいかないので、シミュレーションを使いましょう。
ここではなるべく単純な例として、穴あきバケツの水位を制御することを考えます。

## 水槽のモデル

穴が開いているバケツを考えます。
簡単のためにバケツは円柱または角柱のような形をしているとします。
バケツの底に穴が開いていて、そこから水が漏れて行きます。
またバケツの上部には水を入れるためのバルブがあって、バルブの開き具合を操作することでバケツに水を補充することができるものとします。

<!-- この例における「バルブの開き具合」のような、システムに対する外部からの力や操作を **制御入力** と呼びます。-->

このとき、シミュレーションのためには以下のような情報を考えればよさそうです。

* バケツの断面積 $A$ [m²]
* 時刻 $t$ における水位 $h(t)$ [m]
* 時刻 $t$ においてバケツに注がれる水の量 $Q_{in} (t)$ [m³/s]
* 時刻 $t$ においてバケツから排出される水の量 $Q_{out} (t)$ [m³/s]

そうするとバケツの水位の変化量は以下の式を満たします。

$$
\frac{d h}{d t} (t) = \frac{ Q_{in}(t) - Q_{out} (t) }{ A }
$$

これはバケツの水位 $h$ に対する微分方程式になっています。
初期値を与えれば水位の時間変化をシミュレーションすることができそうですね。
初期値は $h(0) = 0$ としておきましょう。

しかし、まだ未確定のところがあります。
バケツに注がれる水の量 $Q_{in}$ は制御する側が適当に決めればいいですが、バケツから出ていく水の量 $Q_{out}$ はどうやって決まるのでしょうか。
物理の話になりますが、水位 $h(t)$ が高くなるほど排出圧力が増し、流出量が増加すると考えられますからそれを反映させましょう。
ここでは簡単のために、以下の式を満たすと仮定します。

$$
Q_{out} (t) = \sqrt{2 g h(t)}
$$

ただし $g$ は重力加速度 [m/s²] です。

これで情報がそろったので、バケツの数理モデルは完成ですね。

## 微分方程式を数値的に解く

常微分方程式を数値的に解く方法はいくつかあります。
オイラー法[^euler]とか、ルンゲクッタ法とか。
どちらも簡単に実装できるのですが、ここでは解き方は気にせず、ライブラリにお任せすることにします。
複数のライブラリがありますが、とりあえず SciPy の [`solve_ivp`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.integrate.solve_ivp.html#scipy.integrate.solve_ivp) を使用します。
`solve_ivp` は、常微分方程式(ODE)の初期値問題(initial value problem)をいい感じに解いてくれる関数です。

簡単のために、つねに一定量（たとえば 10 [m³/s]）の水が流れ込んでくる想定としてシミュレーションをします。

[^euler]: オイラー法は簡単に実装できてコードが見やすいので例としてよく使われますが、精度が悪いので例としてしか使いません。
